In [0]:
%sql
show databases

databaseName
default
test


In [0]:
%sql
use database default;
show tables;

database,tableName,isTemporary
default,emp_1_csv,false


In [0]:
%sql
--to show DDL
show create table default.emp_1_csv

createtab_stmt
"CREATE TABLE default.emp_1_csv ( EMPNO STRING, ENAME STRING, JOB STRING, MGR STRING, HIREDATE STRING, SAL STRING, COMM STRING, DEPTNO STRING, UPDATED_DATE STRING) USING csv OPTIONS ( 'delimiter' = ',', 'escape' = '""', 'header' = 'true', 'multiLine' = 'false') LOCATION 'dbfs:/FileStore/tables/emp-1.csv'"


In [0]:
#by pyspark to show dbs
spark.sql("show databases").collect()

Out[7]: [Row(databaseName='default'), Row(databaseName='test')]

In [0]:
#to store all dbs into list
alldbs=[db.databaseName for db in spark.sql("show databases").collect()]
alldbs

Out[11]: ['default', 'test']

In [0]:
%sql
create database test

In [0]:
#to show all tables
spark.catalog.listTables('default')

Out[28]: [Table(name='emp_1_csv', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False)]

In [0]:
#to print all tables us forloop with list
alltables=spark.catalog.listTables('default')
for alt in alltables:
    print("table name",alt.name)

table name emp_1_csv


In [0]:
#to show ddls dynamically prepare function
#to print all tables us forloop with list
alltables=spark.catalog.listTables('default')
for alt in alltables:
    ddls=spark.sql("show create table {}.{};".format(database,alt.name));
  


In [0]:
#final function
def createDDL(database):
    alltables = spark.catalog.listTables(database)
    f=open("/tmp/ddls/bkp_{}.sql".format(database),"w")
    for t in alltables:
        ddls = spark.sql("SHOW CREATE TABLE {}.{};".format(database,t.name))
        f.write(ddls.first()[0])
        f.write(";\n")
    f.close()

In [0]:
%fs mkdirs file:/tmp/ddls/

res1: Boolean = true

In [0]:
%fs ls file:/tmp/ddls/

path,name,size,modificationTime
file:/tmp/ddls/bkp_default.sql,bkp_default.sql,332,1701698836317


In [0]:
#call function
createDDL("default")

list file\
verify ddl of table\
%fs ls file:/tmp/ddls/
%fs head file:/tmp/ddls/bkp_default.sql

In [0]:
%fs head file:/tmp/ddls/bkp_default.sql

CREATE TABLE default.emp_1_csv (
 EMPNO STRING,
 ENAME STRING,
 JOB STRING,
 MGR STRING,
 HIREDATE STRING,
 SAL STRING,
 COMM STRING,
 DEPTNO STRING,
 UPDATED_DATE STRING)
USING csv
OPTIONS (
 'delimiter' = ',',
 'escape' = '"',
 'header' = 'true',
 'multiLine' = 'false')
LOCATION 'dbfs:/FileStore/tables/emp-1.csv'
;

In [0]:
#create backup for all databses
alldbs = [db.databaseName for db in spark.sql("show databases").collect()]
alldbs
from multiprocessing.pool import ThreadPool
processes = ThreadPool(4)
processes.map(createDDL,[database for database in alldbs])

Out[45]: [None, None]

In [0]:
%fs ls file:/tmp/ddls/

path,name,size,modificationTime
file:/tmp/ddls/bkp_default.sql,bkp_default.sql,332,1701699362270
file:/tmp/ddls/bkp_test.sql,bkp_test.sql,0,1701699361918
